In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

    # #groq = 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

In [3]:
%pip install langchain-groq
%pip install -qU langchain-groq
%pip install -U langgraph
%pip install spotipy



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
# initialize the state class ;
from langgraph.graph import MessagesState

class SpotifyState(MessagesState):
    selected_playlist_id: str = None  # Store the playlist ID directly
    tool_result: dict = None
    action: str = None
    

In [37]:
#PROMPT TEMPLATES FOR ASSISTANT NODE:
from langchain_core.messages import HumanMessage, SystemMessage 
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

assistant_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant that helps users manage their Spotify playlists."),
    HumanMessage(content="The user wants to add a song to their playlist. Their request is: {user_input}"),
    HumanMessage(content="""Extract the song name and artist name from the user's request.  If you can find them, respond with a JSON object like this:
```
json
{{
  "action": "SearchTrack",
  "arguments": {{
    "song_name": "<song_name>",
    "artist_name": "<artist_name>"
  }}
}} 
```                 
"""),
HumanMessage(content= """ If you cannot find the song and artist, or if the user's request is not about adding a song, respond with a JSON object like this:
```
json
{{
  "action": "RespondToUser",
  "arguments": {{
    "message": "<helpful message to the user>"
    }}
}}
```
"""),
MessagesPlaceholder("messages") ])

In [38]:
# Spotify Authentication
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from langchain_groq import ChatGroq
from langchain.tools import Tool



SCOPE = "playlist-modify-public playlist-read-private playlist-modify-private"
sp_oauth = SpotifyOAuth(
    client_id="4a8fdc631a704303ab465a07d5141a61",  # Replace with your credentials
    client_secret="d649ba5407504a928eb2b94b49fa234d",
    redirect_uri="http://localhost:8888/callback",
    scope=SCOPE,
)
sp = spotipy.Spotify(auth_manager=sp_oauth)

# LLM Setup
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Or your preferred model
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# DEFINING THE TOOL
def search_track_tool(song_name: str, artist_name: str):
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track')
    if results['tracks']['items']:
        track_uri = results['tracks']['items'][0]['uri']
        return {"track_uri": track_uri, "track_name": results['tracks']['items'][0]['name'], "artist_name": results['tracks']['items'][0]['artists'][0]['name']}
    else:
        return None

tools = [
    Tool(
        name="SearchTrack",  # Tool name is case-sensitive!
        func=search_track_tool,
        description="Searches for a track on Spotify given the song name and artist name. Returns a dictionary containing the track URI, track name, and artist name. Returns None if the track is not found.",
    )
]

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [ ]:
# Node Functions
def select_playlist(state: SpotifyState):
    user_input = state.user_input
    results = sp.current_user_playlists()
    for item in results['items']:
        if user_input.lower() in item['name'].lower():
            state.selected_playlist_id = item['id']
            return state
    state.messages.append(HumanMessage(content=f"Playlist '{user_input}' not found."))
    return state

def assistant(state: SpotifyState):
    user_input = state.user_input
    messages = state.messages
    prompt = assistant_prompt.format(user_input=user_input, messages=messages)
    response = llm_with_tools.invoke(prompt)
    messages.append(HumanMessage(content=user_input))
    messages.append(response)

    try:
        json_response = json.loads(response.content)
        state.action = json_response["action"].lower()
        if state.action == "searchtrack": # Extract song and artist only if action is searchtrack
          state.song_name = json_response['arguments']['song_name']
          state.artist_name = json_response['arguments']['artist_name']
        return state
    except (json.JSONDecodeError, KeyError):
        state.action = "respondtouser"
        state.messages.append(HumanMessage(content="I'm having trouble understanding your request."))
        return state

def searchtrack(state: SpotifyState):
    tool_result = search_track_tool(state.song_name, state.artist_name)
    state.tool_result = tool_result
    if tool_result:
        return state
    else:
        state.messages.append(HumanMessage(content=f"Could not find a track matching '{state.song_name}' by '{state.artist_name}'."))
        return state

def add_track(state: SpotifyState):
    tool_result = state.tool_result
    if tool_result:
        try:
            sp.playlist_add_items(state.selected_playlist_id, [tool_result['track_uri']])
            state.messages.append(HumanMessage(content=f"Added '{tool_result['track_name']}' by '{tool_result['artist_name']}' to your playlist."))
            return state
        except Exception as e:
            state.messages.append(HumanMessage(content=f"Error adding track: {e}"))
            return state
    else:
        state.messages.append(HumanMessage(content="No track information found."))
        return state

def respond_to_user(state: SpotifyState):
    return state

def route_action(state: SpotifyState):
    action = state.action
    if action == "searchtrack":
        return state, "searchtrack"
    elif action == "respondtouser":
        return state, "respond_to_user"
    else:
        state.messages.append(HumanMessage(content="Unexpected action received."))
        return state, "respond_to_user"# Node Functions
def select_playlist(state: SpotifyState):
    user_input = state.user_input
    results = sp.current_user_playlists()
    for item in results['items']:
        if user_input.lower() in item['name'].lower():
            state.selected_playlist_id = item['id']
            return state
    state.messages.append(HumanMessage(content=f"Playlist '{user_input}' not found."))
    return state

def assistant(state: SpotifyState):
    user_input = state.user_input
    messages = state.messages
    prompt = assistant_prompt.format(user_input=user_input, messages=messages)
    response = llm_with_tools.invoke(prompt)
    messages.append(HumanMessage(content=user_input))
    messages.append(response)

    try:
        json_response = json.loads(response.content)
        state.action = json_response["action"].lower()
        if state.action == "searchtrack": # Extract song and artist only if action is searchtrack
          state.song_name = json_response['arguments']['song_name']
          state.artist_name = json_response['arguments']['artist_name']
        return state
    except (json.JSONDecodeError, KeyError):
        state.action = "respondtouser"
        state.messages.append(HumanMessage(content="I'm having trouble understanding your request."))
        return state

def searchtrack(state: SpotifyState):
    tool_result = search_track_tool(state.song_name, state.artist_name)
    state.tool_result = tool_result
    if tool_result:
        return state
    else:
        state.messages.append(HumanMessage(content=f"Could not find a track matching '{state.song_name}' by '{state.artist_name}'."))
        return state

def add_track(state: SpotifyState):
    tool_result = state.tool_result
    if tool_result:
        try:
            sp.playlist_add_items(state.selected_playlist_id, [tool_result['track_uri']])
            state.messages.append(HumanMessage(content=f"Added '{tool_result['track_name']}' by '{tool_result['artist_name']}' to your playlist."))
            return state
        except Exception as e:
            state.messages.append(HumanMessage(content=f"Error adding track: {e}"))
            return state
    else:
        state.messages.append(HumanMessage(content="No track information found."))
        return state

def respond_to_user(state: SpotifyState):
    return state

def route_action(state: SpotifyState):
    action = state.action
    if action == "searchtrack":
        return state, "searchtrack"
    elif action == "respondtouser":
        return state, "respond_to_user"
    else:
        state.messages.append(HumanMessage(content="Unexpected action received."))
        return state, "respond_to_user"

In [ ]:
#NODE FUNCTIONS IMPLEMENTED: 


import json

def select_playlist(state, user_input):
    # Search for the playlist ID (replace with your logic)
    results = sp.current_user_playlists()
    playlist_id = None
    for item in results['items']:
        if user_input.lower() in item['name'].lower(): # Case insensitive search
            playlist_id = item['id']
            break

    if playlist_id:
        state["selected_playlist_id"] = playlist_id
        return {}, "assistant"
    else:
        state["messages"] = [HumanMessage(content=f"Playlist '{user_input}' not found.")]
        return {}, "respond_to_user"


def assistant(state):
    user_input = state["user_input"]
    messages = state.get("messages", [])
    prompt = assistant_prompt.format(user_input=user_input, messages=messages)
    response = llm_with_tools.invoke(prompt)
    messages.append(HumanMessage(content=user_input))
    messages.append(response)
    state["messages"] = messages

    try:
        json_response = json.loads(response.content)
        return json_response["arguments"], json_response["action"].lower()
    except (json.JSONDecodeError, KeyError):  # Handle parsing errors
        return {"message": "I'm having trouble understanding your request."}, "respond_to_user"

def searchtrack(state, song_name, artist_name): # Lowercase tool name
    tool_result = search_track_tool(song_name, artist_name)
    state["tool_result"] = tool_result
    state['artis']
    if tool_result:
        return {}, "add_track"
    else:
        return {"message": f"Could not find a track matching '{song_name}' by '{artist_name}'."}, "respond_to_user"

def add_track(state):
    tool_result = state.get("tool_result")
    if tool_result:
        try:
            sp.playlist_add_items(state["selected_playlist_id"], [tool_result['track_uri']])
            return {"message": f"Added '{tool_result['track_name']}' by '{tool_result['artist_name']}' to your playlist."}, "respond_to_user"
        except Exception as e:
            return {"message": f"Error adding track: {e}"}, "respond_to_user" # Handle Spotify API errors
    else:  # Should not happen, but good to check
        return {"message": "No track information found."}, "respond_to_user"


def respond_to_user(state, message):
    state["messages"].append(HumanMessage(content=message)) # Append message to the message history
    return {}, None  # End the graph execution


In [28]:
#ROUTER NODE: 
def route_action(state):
    action = state.get("action")
    if action == "searchtrack":
        return {}, "searchtrack"
    elif action == "respondtouser":
        return {}, "respond_to_user"
    else:  # Handle unexpected actions
        return {"message": "Unexpected action received."}, "respond_to_user"

In [41]:
from langgraph.graph import START, StateGraph , END


builder = StateGraph(SpotifyState)

builder.add_node("select_playlist", select_playlist)
builder.add_node("assistant", assistant)
builder.add_node("route_action", route_action)  # Add the router node
builder.add_node("searchtrack", searchtrack)
builder.add_node("add_track", add_track)
builder.add_node("respond_to_user", respond_to_user)

builder.add_edge("select_playlist", "assistant")
builder.add_edge("assistant", "route_action")  # Always go to router

# Edges from the router are now unconditional. The routing logic is in the node.
builder.add_edge("route_action", "searchtrack")
builder.add_edge("route_action", "respond_to_user")

builder.add_edge("searchtrack", "add_track") # No condition here either.
builder.add_edge("searchtrack", "respond_to_user") # Router node handles the logic now.
builder.add_edge("add_track", "respond_to_user")

builder.set_entry_point("select_playlist")

graph = builder.compile()


In [32]:
#RUN APPROACH 1: 
state = SpotifyState() 
user_input = "Add Mistletoe by Justin Bieber to MyPlaylist"
state["user_input"] = user_input

# Run the agent
result = graph.invoke(state) # Use invoke() method

# Print the final response
print(result["messages"][-1].content)

InvalidUpdateError: Must write to at least one of ['messages', 'selected_playlist_id', 'tool_result', 'action']

In [42]:
messages = [HumanMessage(content="Add Shape of you by Ed Sheeran to my playslist")]
messages = graph.invoke({"messages": messages})

TypeError: select_playlist() missing 1 required positional argument: 'user_input'

In [35]:
user_input = "Add Mistletoe by Justin Bieber to MyPlaylist"
state = SpotifyState(user_input=user_input) 
result = graph.invoke(state)
print(result["messages"][-1].content)  # Access the messages from the returned dict

InvalidUpdateError: Must write to at least one of ['messages', 'selected_playlist_id', 'tool_result', 'action']

In [30]:
# RUN APPROACH 2: 

# Create a simplified invoke function for the Spotify graph
def invoke_spotify_graph(user_input):
    state = SpotifyState()
    state["user_input"] = user_input
    result = graph.invoke(state)
    return result

messages2 = [HumanMessage(content="Add Shape of You by Ed Sheeran to MyPlaylist")]
result2 = invoke_spotify_graph(messages2[0].content)
print(result2['messages'][-1].content)

InvalidUpdateError: Must write to at least one of ['messages', 'selected_playlist_id', 'tool_result', 'action']